# Filter barcodes to remove ones aligning to multiple inserts or second barcode

Arvind Rasi Subramaniam

27 Oct 2021

**Edit this Rscript only in the accompanying .ipynb file. The `snakemake` workflow will automatically export it as a .R script.**

## Load libraries

In [2]:
options(warn = -1)

suppressPackageStartupMessages({

  library(Biostrings)
  library(GenomicAlignments)
  library(plyranges)
  library(tidyverse)

})

## Define analysis-specific variables

In [ ]:
args <- commandArgs(trailingOnly = T)
barcode1_alignment_file <- args[1]
barcode2_alignment_file <- args[2]
barcode_insert_file <- args[3]
read_count_cutoff <- as.numeric(args[4])
output_file <- args[5]

# barcode1_alignment_file <- "../data/ref_vs_ref_alignments/rbp_collision/alignment_barcode1.bam"
# barcode2_alignment_file <- "../data/ref_vs_ref_alignments/rbp_collision/alignment_barcode2.bam"
# barcode_insert_file <- "../data/insert_barcode_counts/rbp_collision.tsv.gz"
# output_file <- "../data/filtered_barcodes/rbp_collision.tsv.gz"

## Read insert-barcode pair counts 

In [ ]:
insert_barcodes <- read_csv(barcode_insert_file) %>% 
  filter(count >= read_count_cutoff) %>%
  print()

## How many barcode1 have multiple inserts or barcode2?

In [ ]:
many_to_one_barcode_combinations <- insert_barcodes %>% 
  group_by(barcode1) %>% 
  mutate(n1 = dplyr::n()) %>% 
  ungroup() %>% 
  group_by(barcode2) %>% 
  mutate(n2 = dplyr::n()) %>% 
  ungroup() %>% 
  filter((n1 > 1) | (n2 > 1)) %>% 
  print()

## Fields to read from BAM file

In [ ]:
# extract the number of mismatches and total edits
param <- ScanBamParam(
  # what = scanBamWhat(),
  what = c("qname", "flag"),
  # extract number of mismatches
  tag = c("XM"), 
  # include only snps; exclude indels
  simpleCigar = T
)

## Read barcode vs barcode alignments for barcodes 1

In [ ]:
bamfile1 <- BamFile(barcode1_alignment_file)
alns1 <- readGAlignments(bamfile1, param = param) %>% 
  as_tibble() %>% 
  mutate(rname = as.character(seqnames)) %>% 
  select(rname, qname, flag, XM) %>% 
  type_convert() %>% 
  print()

## Read barcode vs barcode alignments for barcodes 2

In [ ]:
bamfile2 <- BamFile(barcode2_alignment_file)
alns2 <- readGAlignments(bamfile2, param = param) %>% 
  as_tibble() %>% 
  mutate(rname = as.character(seqnames)) %>% 
  select(rname, qname, flag, XM) %>% 
  type_convert() %>% 
  print()

## Find barcode1 that are linked to distinct insert or might be sequencing errors

In [ ]:
exclude1 <- alns1 %>% 
  filter(rname != qname) %>%
  left_join(select(insert_barcodes, insert_num, barcode_num, count), by = c("rname" = "barcode_num")) %>%
  rename(rinsert = insert_num, rcount = count) %>%
  right_join(select(insert_barcodes, insert_num, barcode_num, count), by = c("qname" = "barcode_num")) %>%
  rename(qinsert = insert_num, qcount = count) %>%
  # this exludes:
  # 1. barcodes that map to two distinct inserts
  # 2. barcodes that got lower count than another homologous barcode with same insert
  filter(!(qinsert == rinsert & qcount > rcount)) %>%
  arrange(qname) %>% 
  distinct(qname) %>%
  print()

## Find barcode2 that are linked to distinct insert or might be sequencing errors

In [ ]:
exclude2 <- alns2 %>% 
  filter(rname != qname) %>%
  left_join(select(insert_barcodes, insert_num, barcode_num, count), by = c("rname" = "barcode_num")) %>%
  rename(rinsert = insert_num, rcount = count) %>%
  right_join(select(insert_barcodes, insert_num, barcode_num, count), by = c("qname" = "barcode_num")) %>%
  rename(qinsert = insert_num, qcount = count) %>%
  # this exludes:
  # 1. barcodes that map to two distinct inserts
  # 2. barcodes that got lower count than another homologous barcode with same insert
  filter(!(qinsert == rinsert & qcount > rcount)) %>%
  arrange(qname) %>% 
  distinct(qname) %>%
  print()

## Write barcodes that do not clash to output

In [ ]:
filtered_barcodes <- insert_barcodes %>% 
  anti_join(select(exclude1, qname), by = c("barcode_num" = "qname")) %>%
  anti_join(select(exclude2, qname), by = c("barcode_num" = "qname")) %>%
  anti_join(select(many_to_one_barcode_combinations, barcode_num), by = "barcode_num") %>%
  mutate(barcode2 = as.character(reverseComplement(DNAStringSet(barcode2)))) %>%
  select(insert_num, barcode_num, barcode1, barcode2, count) %>%
  arrange(desc(count)) %>%
  rename(linkage_count = count) %>%
  mutate(barcode_num = 1:dplyr::n()) %>%
  write_csv(output_file) %>%
  print()